In [1]:
import numpy as np
from tensorflow.keras.datasets import mnist
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt

# Définition des fonctions d'activation et de perte
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    return x * (1 - x)

def softmax(x):
    exps = np.exp(x - np.max(x))
    return exps / np.sum(exps, axis=1, keepdims=True)

def categorical_crossentropy(y_true, y_pred):
    return -np.mean(np.sum(y_true * np.log(y_pred), axis=1))

# Définition du réseau CNN
class CNN:
    def __init__(self):
        self.conv_weights = np.random.randn(3, 3, 1, 8)
        self.conv_bias = np.zeros((1, 8))
        self.fc_weights = np.random.randn(13*13*8, 10)
        self.fc_bias = np.zeros((1, 10))

    def forward(self, X):
        # Convolution
        self.conv_output = np.zeros((X.shape[0], 26, 26, 8))
        for i in range(X.shape[0]):
            for j in range(26):
                for k in range(26):
                    self.conv_output[i, j, k] = sigmoid(np.sum(X[i, j:j+3, k:k+3] * self.conv_weights) + self.conv_bias)

        # Flatten
        self.flatten_output = self.conv_output.reshape(X.shape[0], -1)

        # Fully connected layer
        self.fc_output = softmax(np.dot(self.flatten_output, self.fc_weights) + self.fc_bias)
        return self.fc_output

    def backward(self, X, y, lr=0.01):
        # Compute gradients for fully connected layer
        fc_grad = (self.fc_output - y) / y.shape[0]
        self.fc_weights_grad = np.dot(self.flatten_output.T, fc_grad)
        self.fc_bias_grad = np.sum(fc_grad, axis=0, keepdims=True)

        # Compute gradients for convolutional layer
        conv_grad = np.dot(fc_grad, self.fc_weights.T) * sigmoid_derivative(self.flatten_output)
        conv_grad = conv_grad.reshape(X.shape[0], 26, 26, 8)

        self.conv_weights_grad = np.zeros_like(self.conv_weights)
        self.conv_bias_grad = np.zeros_like(self.conv_bias)
        for i in range(X.shape[0]):
            for j in range(26):
                for k in range(26):
                    self.conv_weights_grad += X[i, j:j+3, k:k+3] * conv_grad[i, j, k][:, np.newaxis]
                    self.conv_bias_grad += conv_grad[i, j, k]

        # Update weights and biases
        self.fc_weights -= lr * self.fc_weights_grad
        self.fc_bias -= lr * self.fc_bias_grad
        self.conv_weights -= lr * self.conv_weights_grad
        self.conv_bias -= lr * self.conv_bias_grad

# Chargement des données MNIST
mndata = mnist('./data')
images, labels = mndata.load_training()

# Prétraitement des données
X = np.array(images).reshape(-1, 28, 28) / 255.0
y = np.zeros((len(labels), 10))
y[np.arange(len(labels)), labels] = 1

# Création des ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entraînement du modèle
cnn = CNN()
epochs = 10
batch_size = 32
for epoch in range(epochs):
    for i in range(0, len(X_train), batch_size):
        batch_X, batch_y = X_train[i:i+batch_size], y_train[i:i+batch_size]
        cnn.forward(batch_X)
        cnn.backward(batch_X, batch_y)

# Inférence sur le dataset de test
y_pred = np.argmax(cnn.forward(X_test), axis=1)

# Calcul de l'accuracy
accuracy = accuracy_score(np.argmax(y_test, axis=1), y_pred)
print("Accuracy:", accuracy)

# Affichage de la matrice de confusion
conf_matrix = confusion_matrix(np.argmax(y_test, axis=1), y_pred)
print("Confusion Matrix:")
print(conf_matrix)

# Affichage de l'évolution de l'accuracy pendant l'entraînement
plt.plot(range(1, epochs + 1), accuracies)
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training Accuracy')
plt.show()


1/100, error=0.8638013354839185
2/100, error=0.7891108005904128
3/100, error=0.7317853815692537
4/100, error=0.6664389063188166
5/100, error=0.576806142952728
6/100, error=0.4562971807542781
7/100, error=0.32270389158268115
8/100, error=0.2219257093713864
9/100, error=0.17933398004809997
10/100, error=0.15558950314599265
11/100, error=0.13792174201113133
12/100, error=0.12452729976127111
13/100, error=0.11684951739581233
14/100, error=0.11119095654853957
15/100, error=0.10521897153673741
16/100, error=0.10004776335819313
17/100, error=0.09614648117070043
18/100, error=0.09396690857192916
19/100, error=0.09224659987729432
20/100, error=0.09090516102168465
21/100, error=0.0896097728185031
22/100, error=0.08839353231072047
23/100, error=0.08721808108541591
24/100, error=0.08604226702130048
25/100, error=0.08495052117503411
26/100, error=0.083895692413635
27/100, error=0.08289689671005473
28/100, error=0.08204761591631617
29/100, error=0.08121164546221078
30/100, error=0.08035926345422115


ImportError: cannot import name 'MNIST' from 'mnist' (c:\Users\Serval T. Nono\Documents\Mon Master 2\Data Science\TPE-CC\datascince-groupe6\2e TPE CNN\Neural-Network-master\mnist.py)